In [5]:
import torch
import catboost
from torch import nn
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import TensorDataset,DataLoader
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import warnings 
warnings.filterwarnings('ignore')
gc.collect()

29

In [6]:
def backbone(self, x):
    x = self.module.conv1(x)
    x = self.module.bn1(x)
    x = self.module.relu(x)

    x = self.module.layer1(x)
    x = self.module.layer2(x)
    x = self.module.layer3(x)

    return x

def preprocessing_Y(file_path):
    Y = pd.read_csv(file_path).iloc[:,:]
    Y = Y[Y.Filename != 'train_01046']
    enc = OneHotEncoder().fit(Y[['Label']])
    Y_one_hot = enc.transform(Y[['Label']]).toarray()
    Y_one_hot = torch.FloatTensor(Y_one_hot)
    print('Y_ont_hot shape',Y_one_hot.shape)
    print('Y_df shape',Y.shape)
    return Y_one_hot,Y

def load_pretrain_senet(model_path):
    model = torch.hub.load(
        'moskomule/senet.pytorch',
        'se_resnet20',
        num_classes=6)
    model.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.fc = nn.Sequential(
    nn.Linear(64,64),
    nn.LeakyReLU(),
    nn.Dropout(0.5),#三個臭皮匠勝過一個諸葛亮
    nn.Linear(64,6))
    model = torch.nn.DataParallel(model, device_ids=[0, 1, 2 ,3])
    model.load_state_dict(torch.load(model_path))
    model.eval()
    model.backbone = backbone.__get__(model)
    return model

def get_senet_output(senet,data):
    return senet(data)

def get_all_senet_output(data):
    y = senet(data)
    return y.detach().cpu().numpy()

def get_X_numpy(X):
    X_train_np = np.array([[]])
    data_iter = DataLoader(TensorDataset(torch.FloatTensor(X.to(torch.float32))),batch_size=256)
    for bx in tqdm(data_iter):
        bx = bx[0]
        bx = bx.to('cuda:0')
        y_hat = get_all_senet_output(bx)
        if len(X_train_np) == 1:
            X_train_np = y_hat
        else:
            X_train_np = np.vstack((X_train_np,y_hat))
    return X_train_np

def get_X_numpy_backbone(X):
    X_train_np = np.array([[]])
    data_iter = DataLoader(TensorDataset(torch.FloatTensor(X.to(torch.float32))),batch_size=64)
    for bx in tqdm(data_iter):
        bx = bx[0]
        bx = bx.to('cuda:0')
        y_hat = senet.backbone(bx).detach().cpu().numpy()
        y_hat = y_hat.reshape(-1,64*20*40)
        if len(X_train_np) == 1:
            X_train_np = y_hat
        else:
            X_train_np = np.vstack((X_train_np,y_hat))
    return X_train_np

# 路徑

In [7]:
Y_train_path = 'train/meta_train.csv'
senet = 'senet20_20210605_random_state529_validacc_0.902418682235196用訓練資料1199當驗證集.pt'

# load k 個 senet

In [8]:
senet = load_pretrain_senet(senet).to('cuda:0')
senet

Using cache found in /root/.cache/torch/hub/moskomule_senet.pytorch_master


DataParallel(
  (module): CifarSEResNet(
    (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): CifarSEBasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (se): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)
          (fc): Sequential(
            (0): Linear(in_features=16, out_features=1, bias=False)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=1, out_features=16, bias=False

# 訓練 和 測試資料

In [9]:
X_train_all = torch.load('X_train_吳啟聖教的方法.pt')
X_test_all = torch.load('X_test_吳啟聖教的方法.pt')
Y_train_all,Y_train_df = preprocessing_Y(Y_train_path)
map_dict = {}
for l in Y_train_df.Label.unique():
    map_dict[l] = Y_train_df[Y_train_df.Label==l].sample(1)['Remark'].values[0]
map_dict

Y_ont_hot shape torch.Size([1199, 6])
Y_df shape (1199, 3)


{0: 'Barking',
 1: 'Howling',
 2: 'Crying',
 3: 'COSmoke',
 4: 'GlassBreaking',
 5: 'Vacuum'}

In [11]:
Y_test_all = pd.read_csv('vote_V6_排行榜0.975345284.csv') #拿目前分數最高的
Y_test_all.head(5)
values = Y_test_all.iloc[:,1:].values.argmax(axis=1) #轉換成one hot格式
n_values = np.max(values) + 1
np.eye(n_values)[values][:5]
Y_test_all.iloc[:,1:] = np.eye(n_values)[values] #轉換成df格式
Y_test_all
Y_test_all = torch.FloatTensor(Y_test_all.iloc[:10000,1:].values) #轉換成tensor格式
Y_test_all,Y_test_all.shape

(tensor([[0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0.],
         ...,
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.]]), torch.Size([10000, 6]))

In [17]:
from sklearn.model_selection import train_test_split

#X_train, X_valid , y_train, y_valid = train_test_split(X_train_all, Y_train_all, 
#                                                       test_size=0.3, 
#                                                       random_state=84,
#                                                       stratify=Y_train_all)

X_train = X_test_all
X_valid = X_train_all
y_train = Y_test_all
y_valid = Y_train_all
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(pd.DataFrame(y_train.argmax(axis=1)).value_counts())
print(pd.DataFrame(y_valid.argmax(axis=1)).value_counts())

torch.Size([10000, 1, 80, 157])
torch.Size([10000, 6])
torch.Size([1199, 1, 80, 157])
torch.Size([1199, 6])
5    3556
2    2152
1    1716
0    1264
3     869
4     443
dtype: int64
0    200
1    200
2    200
3    200
5    200
4    199
dtype: int64


In [18]:
from torch.utils.data import TensorDataset,DataLoader
trainset = TensorDataset(torch.FloatTensor(X_train.to(torch.float32)),torch.FloatTensor(y_train))
vaildset = TensorDataset(torch.FloatTensor(X_valid.to(torch.float32)),torch.FloatTensor(y_valid))
train_iter = DataLoader(trainset,batch_size=32,num_workers=4)
vaild_iter = DataLoader(vaildset,batch_size=32,num_workers=4)

In [19]:
device = 'cuda:0'
for bx,by in train_iter:
    print(bx.shape,bx.device) # batch,channel,freq,time
    print(by.shape,by.device) # batch,n_class
    print(senet.backbone(bx.to(device)).size())
    break

torch.Size([32, 1, 80, 157]) cpu
torch.Size([32, 6]) cpu
torch.Size([32, 64, 20, 40])


In [22]:
from sklearn.metrics import confusion_matrix,accuracy_score
def plot_confusion_matrix(model,data_iter,map_dict=map_dict):
    y_true = np.array([])
    y_pred = np.array([])
    for bx,by in tqdm(data_iter):
        bx = bx.to(device)
        by = by.to(device)
        y_hat = model(bx)
        try:
            if len(y_true) == 0:
                y_true = by.argmax(axis=1).detach().cpu().numpy()
                y_pred = y_hat.argmax(axis=1).detach().cpu().numpy()
            else:
                y_true = np.hstack((y_true,by.argmax(axis=1).detach().cpu().numpy()))
                y_pred = np.hstack((y_pred,y_hat.argmax(axis=1).detach().cpu().numpy()))
        except:
            pass
    cm = pd.DataFrame(confusion_matrix(y_pred,y_true))
    cm.columns = list(map_dict.values())
    acc = accuracy_score(y_pred,y_true)
    return cm,acc

In [25]:
model = senet
model.eval()
cm,acc = plot_confusion_matrix(model.to(device),train_iter)
print(acc)
cm

100%|██████████| 313/313 [00:23<00:00, 13.27it/s]

0.9046


,Barking,Howling,Crying,COSmoke,GlassBreaking,Vacuum
0,1084,73,34,1,45,86
1,21,1447,21,1,1,19
2,75,138,2047,16,0,81
3,8,22,8,837,4,20
4,12,0,0,5,322,41
5,64,36,42,9,71,3309


In [26]:
cm,acc = plot_confusion_matrix(model.to(device),vaild_iter)
print(acc)
cm

100%|██████████| 38/38 [00:01<00:00, 20.29it/s]

0.902418682235196


,Barking,Howling,Crying,COSmoke,GlassBreaking,Vacuum
0,175,3,4,2,16,0
1,6,168,15,0,0,1
2,13,25,175,1,0,5
3,0,1,1,194,0,0
4,1,0,0,1,181,5
5,5,3,5,2,2,189


# 有辦法在提升驗證集的表現嗎?

# 接 LogisticRegression 修正senet的弱點

In [29]:
X_train_np = get_X_numpy(X_train)
y_train_np = y_train.detach().numpy().argmax(axis=1)
X_valid_np = get_X_numpy(X_valid)
y_valid_np = y_valid.detach().numpy().argmax(axis=1)
lg = LogisticRegression()
lg.fit(X_train_np,y_train_np)
y_true = y_valid_np
y_pred = lg.predict(X_valid_np )
print(accuracy_score(y_pred,y_true))
cm = pd.DataFrame(confusion_matrix(y_pred,y_true))
cm.columns = list(map_dict.values())
cm

100%|██████████| 5/5 [00:00<00:00, 25.59it/s]


0.9074228523769808


,Barking,Howling,Crying,COSmoke,GlassBreaking,Vacuum
0,177,2,5,2,11,0
1,9,180,25,0,1,1
2,8,15,164,1,0,5
3,1,0,0,194,0,0
4,2,0,0,1,184,5
5,3,3,6,2,3,189


# cma-es優化 LogisticRegression

In [30]:
import optuna
sampler = optuna.samplers.CmaEsSampler()
study = optuna.create_study(sampler=sampler,direction='maximize')
search_iteration = 100
for i in tqdm(range(search_iteration)):
    trial = study.ask()
    class_weight = {}
    for i in [0,1,2,3,4,5]:
        class_weight[i] = trial.suggest_uniform(i,0,1)
    lg = LogisticRegression(class_weight=class_weight)
    lg.fit(X_train_np,y_train_np)
    y_true = y_valid_np
    y_pred = lg.predict(X_valid_np)
    acc = accuracy_score(y_pred,y_true)
    study.tell(trial,acc)
    
study.best_params

[I 2021-06-07 00:56:15,833] A new study created in memory with name: no-name-ed42a4c3-84ab-4293-82bf-de243608c24a
100%|██████████| 100/100 [02:44<00:00,  1.64s/it]


{0: 0.3095747062346247,
 1: 0.8211889048835475,
 2: 0.8232005317028085,
 3: 0.6012470372442934,
 4: 0.6656270922649381,
 5: 0.4571863612662681}

In [31]:
X_train_np = get_X_numpy(X_train)
y_train_np = y_train.detach().numpy().argmax(axis=1)
X_valid_np = get_X_numpy(X_valid)
y_valid_np = y_valid.detach().numpy().argmax(axis=1)
lg = LogisticRegression(class_weight=study.best_params)
lg.fit(X_train_np,y_train_np)
y_true = y_valid_np
y_pred = lg.predict(X_valid_np)
acc = accuracy_score(y_pred,y_true)
print(acc)
cm = pd.DataFrame(confusion_matrix(y_pred,y_true))
cm.columns = list(map_dict.values())
cm

100%|██████████| 5/5 [00:00<00:00, 26.38it/s]


0.9107589658048374


,Barking,Howling,Crying,COSmoke,GlassBreaking,Vacuum
0,173,1,4,2,7,0
1,9,182,25,0,1,1
2,12,14,166,1,0,5
3,1,0,0,194,0,0
4,2,0,0,1,188,5
5,3,3,5,2,3,189


In [32]:
final_prob = lg.predict_proba(X_train_np)

In [33]:
final_prob.sum(axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

In [34]:
sample_submit = pd.read_csv('sample_submission.csv')
sample_submit.iloc[:10000,1:] = final_prob
sample_submit

,Filename,Barking,Howling,Crying,COSmoke,GlassBreaking,Other
0,public_00001,2.656513e-05,0.000040,0.000191,3.925591e-06,0.003297,0.996441
1,public_00002,1.079857e-07,0.000003,0.000004,1.112289e-11,0.000002,0.999991
2,public_00003,9.948332e-01,0.002993,0.001376,4.177518e-07,0.000714,0.000084
3,public_00004,1.432061e-05,0.000006,0.000205,1.032362e-07,0.000653,0.999121
4,public_00005,3.056253e-01,0.149063,0.534803,1.195399e-04,0.000339,0.010051
...,...,...,...,...,...,...,...
29995,private_19996,1.666000e-01,0.166600,0.166600,1.666000e-01,0.166600,0.167000
29996,private_19997,1.666000e-01,0.166600,0.166600,1.666000e-01,0.166600,0.167000
29997,private_19998,1.666000e-01,0.166600,0.166600,1.666000e-01,0.166600,0.167000
29998,private_19999,1.666000e-01,0.166600,0.166600,1.666000e-01,0.166600,0.167000


In [35]:
acc

0.9107589658048374

In [36]:
sample_submit.to_csv(f'submit_valid_acc_{acc}_senet_1199筆當驗證_接logistic做最後修正.csv',index=False)
print('done')

done
